In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
import scipy.io
from TrainAndEvaluate import train_model, evaluate_model, ECGDataset
from MuDANet_v1 import MuDANet

# Hyperparameter

In [2]:
lr = 0.0001
batch_size = 32
num_epochs = 40
num_classes = 3
gpu_num = "0"

# 일반 학습

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv1d):
        nn.init.xavier_uniform_(m.weight)

    
# 저장된 데이터 불러오기
train_data = torch.load('../data/train_data_MuDANet.pt')
val_data = torch.load('../data/val_data_MuDANet.pt')
train_labels = torch.load('../data/train_labels_MuDANet.pt')
val_labels = torch.load('../data/val_labels_MuDANet.pt')

# 데이터셋 생성
train_dataset = ECGDataset(train_data, train_labels)
val_dataset = ECGDataset(val_data, val_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = MuDANet(num_classes=num_classes)
model.apply(weights_init)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

device = torch.device("cuda:" + gpu_num if torch.cuda.is_available() else "cpu")

tr_loss, tr_acc, val_loss, val_acc = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

Epoch [1/40], Loss: 0.9199, Accuracy: 0.6294
Validation Loss: 0.8696, Validation Accuracy: 0.6720, time: 34.66385817527771
Epoch [2/40], Loss: 0.8732, Accuracy: 0.6727
Validation Loss: 0.8407, Validation Accuracy: 0.7000, time: 36.111063718795776
Epoch [3/40], Loss: 0.8392, Accuracy: 0.7058
Validation Loss: 0.8222, Validation Accuracy: 0.7243, time: 29.884281396865845
Epoch [4/40], Loss: 0.8226, Accuracy: 0.7228
Validation Loss: 0.8026, Validation Accuracy: 0.7433, time: 35.83068346977234
Epoch [5/40], Loss: 0.8129, Accuracy: 0.7353
Validation Loss: 0.7866, Validation Accuracy: 0.7637, time: 37.07797408103943
Epoch [6/40], Loss: 0.8045, Accuracy: 0.7425
Validation Loss: 0.7865, Validation Accuracy: 0.7619, time: 34.07093095779419
Epoch [7/40], Loss: 0.7994, Accuracy: 0.7482
Validation Loss: 0.7729, Validation Accuracy: 0.7763, time: 28.02358913421631
Epoch [8/40], Loss: 0.7927, Accuracy: 0.7549
Validation Loss: 0.7676, Validation Accuracy: 0.7797, time: 19.814042806625366
Epoch [9/40],

In [ ]:
import matplotlib.pyplot as plt

plt.plot(tr_loss, label='train loss')
plt.plot(val_loss, label='validation loss')
plt.legend()

plt.show()

plt.plot(tr_acc, label='train accuracy')
plt.plot(val_acc, label='validation accuracy')
plt.legend()

plt.show()

# K - Fold

In [ ]:
fold_accuracy = []
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=30, device='cuda'):
    model = model.to(device)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}')

        val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, device)
        print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')
        fold_accuracy.append(val_accuracy)


def evaluate_model(model, val_loader, criterion, device='cuda'):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct / total
    return val_loss, val_accuracy


class ECGDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def weights_init(m):
    if isinstance(m, nn.Conv1d):
        nn.init.xavier_uniform_(m.weight)

def main():

    np_data = np.load('/content/drive/MyDrive/ECG/training_9s.npy').astype(np.float32)
    label = np.load('/content/drive/MyDrive/ECG/training_9s_label.npy').astype(np.float32)
    X_tensor = torch.tensor(np_data.reshape(np_data.shape[0], 1, np_data.shape[2]), dtype=torch.float32)
    y_tensor = torch.tensor(label, dtype=torch.long)

    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    for fold, (train_index, test_index) in enumerate(kf.split(X_tensor)):
        print(f"Fold {fold + 1}")
        # Train/test 데이터 나누기
        X_train, X_test = X_tensor[train_index], X_tensor[test_index]
        y_train, y_test = y_tensor[train_index], y_tensor[test_index]

        # 데이터 로더 생성
        train_dataset = ECGDataset(X_train, y_train)
        test_dataset = ECGDataset(X_test, y_test)

        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

        model = MuDANet(num_classes=3)
        model.apply(weights_init)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.0001)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=40, device=device)



if __name__ == '__main__':
    main()
    print(f"\nOverall Accuracy: {np.mean(fold_accuracy):.4f}")